# Creating a frequency finder auto-encoder

## Import Libraries

In [17]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from helpers import *
import os
import getpass
import subprocess

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get drive path 

In [19]:
drive_path = find_drive_path()
drive_path

'/media/mat2m10/Samsung_T5'

In [20]:
data = pd.read_pickle(f"{drive_path}/Data/Inputs/Synthetisch/Syn_tot.pkl")
data.head()

,IID,Pheno,Geno,State
0,0pop1,1,"[2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, ...",pop1sick
1,1pop1,1,"[2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, ...",pop1sick
2,2pop1,1,"[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, ...",pop1sick
3,3pop1,0,"[2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",pop1healthy
4,4pop1,1,"[2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, ...",pop1sick


In [21]:
data.tail()

,IID,Pheno,Geno,State
1995,995pop2,0,"[2, 0, 0, 0, 1, 2, 2, 2, 1, 0, 1, 0, 2, 1, 1, ...",pop2healthy
1996,996pop2,0,"[1, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, ...",pop2healthy
1997,997pop2,0,"[1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 2, 1, 2, ...",pop2healthy
1998,998pop2,0,"[2, 2, 1, 2, 0, 1, 0, 2, 0, 1, 0, 2, 2, 0, 0, ...",pop2healthy
1999,999pop2,0,"[1, 0, 1, 1, 2, 1, 1, 0, 0, 0, 2, 1, 1, 1, 1, ...",pop2healthy


In [22]:
print(tf.__version__)

2.5.1


In [5]:
import tensorflow as tf

In [4]:
tf.config.list_physical_devices("GPU")

[]